In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/afml-assignment-1/final/41_AlgoNauts/test-part2.txt
/kaggle/input/afml-assignment-1/final/41_AlgoNauts/train-part2.csv
/kaggle/input/afml-assignment-1/final/41_AlgoNauts/test-part1.csv
/kaggle/input/afml-assignment-1/final/41_AlgoNauts/train-part1-noise.csv
/kaggle/input/afml-assignment-1/final/41_AlgoNauts/boilerplate.ipynb
/kaggle/input/afml-assignment-1/final/41_AlgoNauts/train-part1-clean.csv
/kaggle/input/afml-assignment-1/final/29_Gradient Alchemists/test-part2.txt
/kaggle/input/afml-assignment-1/final/29_Gradient Alchemists/train-part2.csv
/kaggle/input/afml-assignment-1/final/29_Gradient Alchemists/test-part1.csv
/kaggle/input/afml-assignment-1/final/29_Gradient Alchemists/train-part1-noise.csv
/kaggle/input/afml-assignment-1/final/29_Gradient Alchemists/boilerplate.ipynb
/kaggle/input/afml-assignment-1/final/29_Gradient Alchemists/train-part1-clean.csv
/kaggle/input/afml-assignment-1/final/20_Biasically Us/test-part2.txt
/kaggle/input/afml-assignment-1/final/20_B

# Import Requiremnts 

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.fft import fft, fftfreq

# Understanding the Noise and Bssic EDA of Weights

In [3]:
clean = pd.read_csv('/kaggle/input/afml-assignment-1/final/24_opacus/train-part1-clean.csv', header=None).values.flatten()
noisy_train = pd.read_csv('/kaggle/input/afml-assignment-1/final/24_opacus/train-part1-noise.csv', header=None).values.flatten()
noisy_test = pd.read_csv('/kaggle/input/afml-assignment-1/final/24_opacus/test-part1.csv', header=None).values.flatten()


In [4]:
clean = clean.reshape(-1, 20)
noisy_train = noisy_train.reshape(-1, 20)
noisy_test = noisy_test.reshape(-1, 20)

In [5]:
noise = noisy_train - clean

In [6]:
# Noise statistics 
print(f"   Mean: {noise.mean():.6f}")
print(f"   Std: {noise.std():.6f}")
print(f"   Min: {noise.min():.6f}")
print(f"   Max: {noise.max():.6f}")
print(f"   Median: {np.median(noise):.6f}")

   Mean: 0.878631
   Std: 810.585416
   Min: -0.660046
   Max: 3168884.810232
   Median: 0.035964


In [7]:
# tekks us if noise is additive or multiplicative 
correlation = np.corrcoef(clean.flatten(), noise.flatten())[0, 1]
if abs(correlation) < 0.1:
    print("Additive (Independednt of the signal)")
else:
    print("Multiplicative or signal dependent ")

Additive (Independednt of the signal)


In [8]:
_, p_value_gaussian = stats.normaltest(noise.flatten())
print(f"Gaussian Test (p-value): {p_value_gaussian:.6f}")
if p_value_gaussian > 0.05:
    print(" Noise is likely to be guassian")
else:
    print("Not likely to be guassian")

Gaussian Test (p-value): 0.000000
Not likely to be guassian


In [9]:
skewness = stats.skew(noise.flatten())
kurtosis = stats.kurtosis(noise.flatten())
print(skewness)
print(kurtosis)

3132.988314990205
11655238.65809521


# Main Denoiser Code 


In Previous Submmsions We have tried AutoEncoders and HuberLoss and Other techniques through discussions we have found out Huber Regressor and Ridge Regressor gives us the best Output

From the EDA we get to know the dostribution of our dataset and the strong outlier hence we try to use Hubber Regressor and Ridge Regressor 

In [10]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge, HuberRegressor
from sklearn.preprocessing import PowerTransformer, RobustScaler, QuantileTransformer
import time

In [11]:
clean = pd.read_csv('/kaggle/input/afml-assignment-1/final/24_opacus/train-part1-clean.csv', header=None).values.flatten()
noisy = pd.read_csv('/kaggle/input/afml-assignment-1/final/24_opacus/train-part1-noise.csv', header=None).values.flatten()

In [12]:
clean = clean.reshape(-1,20)
noisy = noisy.reshape(-1, 20)

In [13]:
quantile_X = QuantileTransformer(n_quantiles=2000, output_distribution='normal', random_state=42)
quantile_y = QuantileTransformer(n_quantiles=2000, output_distribution='normal', random_state=42)


In [14]:
X_quantile = quantile_X.fit_transform(noisy)
y_quantile = quantile_y.fit_transform(clean)

In [15]:
scaler_X = RobustScaler(quantile_range=(10, 90))
scaler_y = RobustScaler(quantile_range=(10, 90))
X_scaled = scaler_X.fit_transform(X_quantile)
y_scaled = scaler_y.fit_transform(y_quantile)


In [16]:
power_transformer_X = PowerTransformer(method='yeo-johnson', standardize=True)
power_transformer_y = PowerTransformer(method='yeo-johnson', standardize=True)

X_train = power_transformer_X.fit_transform(X_scaled)
y_train = power_transformer_y.fit_transform(y_scaled)

print(f"Transformed X shape: {X_train.shape}")
print(f"Transformed y shape: {y_train.shape}")

Transformed X shape: (1024147, 20)
Transformed y shape: (1024147, 20)


# Training 

In [17]:
tart_time = time.time()

huber_models = []
ridge_models = []

for i in range(20):
    huber = HuberRegressor(
        epsilon=1.05,       
        alpha=0.0001,      
        max_iter=500,
        tol=1e-6,
        fit_intercept=True
    )
    huber.fit(X_train, y_train[:, i])
    huber_models.append(huber)
    
    
    ridge = Ridge(
        alpha=0.001,       
        fit_intercept=True,
        max_iter=3000,
        tol=1e-7,
        solver='auto'
    )
    ridge.fit(X_train, y_train[:, i])
    ridge_models.append(ridge)

In [18]:
test = pd.read_csv('/kaggle/input/afml-assignment-1/final/24_opacus/test-part1.csv', header=None, skiprows=1).values.astype(np.float32)

print(f"Test data shape: {test.shape}")
test_quantile = quantile_X.transform(test)
test_scaled = scaler_X.transform(test_quantile)
test_transformed = power_transformer_X.transform(test_scaled)


Test data shape: (12732, 20)


Ensambling Both the models 

In [19]:
huber_preds = np.zeros((test_transformed.shape[0], 20))
ridge_preds = np.zeros((test_transformed.shape[0], 20))

for i in range(20):
    huber_preds[:, i] = huber_models[i].predict(test_transformed)
    ridge_preds[:, i] = ridge_models[i].predict(test_transformed)


huber_weight = 0.7
ridge_weight = 0.3

predictions_transformed = huber_weight * huber_preds + ridge_weight * ridge_preds


predictions_scaled = power_transformer_y.inverse_transform(predictions_transformed)
predictions_robust = scaler_y.inverse_transform(predictions_scaled)
predictions_final = quantile_y.inverse_transform(predictions_robust)
print(f"Predictions shape: {predictions_final.shape}")
print(f"Predictions range: [{predictions_final.min():.6f}, {predictions_final.max():.6f}]")

Predictions shape: (12732, 20)
Predictions range: [-0.471118, 0.131664]


Submmision File For Part 2 

In [20]:
df_out = pd.DataFrame(predictions_final, columns=[f'Col{i+1}' for i in range(20)])
df_out.insert(0, 'id', range(1, len(predictions_final) + 1))

df_out.to_csv('submission_best.csv', index=False)
print("Submmsion_csv has been processed")
print(f"Submission saved: {df_out.shape}")

Submmsion_csv has been processed
Submission saved: (12732, 21)


Validation on training data (for performance metrics)

In [21]:
huber_train_preds = np.zeros((X_train.shape[0], 20))
ridge_train_preds = np.zeros((X_train.shape[0], 20))

for i in range(20):
    huber_train_preds[:, i] = huber_models[i].predict(X_train)
    ridge_train_preds[:, i] = ridge_models[i].predict(X_train)

train_pred_transformed = huber_weight * huber_train_preds + ridge_weight * ridge_train_preds
train_pred_scaled = power_transformer_y.inverse_transform(train_pred_transformed)
train_pred_robust = scaler_y.inverse_transform(train_pred_scaled)
train_pred = quantile_y.inverse_transform(train_pred_robust)

mse = np.mean((train_pred - clean)**2)
mae = np.mean(np.abs(train_pred - clean))
rmse = np.sqrt(mse)


In [22]:
print(mse)
print(rmse)
print(mae)

0.008040021529692671
0.08966616714063712
0.031207547421272727


Indivisual Performance of each ensamble that we used

In [23]:
huber_train_final = quantile_y.inverse_transform(scaler_y.inverse_transform(power_transformer_y.inverse_transform(huber_train_preds)))
ridge_train_final = quantile_y.inverse_transform(scaler_y.inverse_transform(power_transformer_y.inverse_transform(ridge_train_preds)))

huber_mae = np.mean(np.abs(huber_train_final - clean))
ridge_mae = np.mean(np.abs(ridge_train_final - clean))


In [24]:
print(huber_mae)
print(ridge_mae)
print(mae)

0.02999963368580386
0.033193327567791094
0.031207547421272727
